In [60]:
import torch.utils.data as data

from PIL import Image
import os
import numpy as np
from numpy.random import randint


class VideoRecord(object):
    def __init__(self, row):
        self._data = row

    @property
    def path(self):
        return self._data[0]

    @property
    def num_frames(self):
        return int(self._data[1])

    @property
    def label(self):
        return int(self._data[2])


class TSNDataSet(data.Dataset):
    def __init__(self, root_path, list_file,
                 num_segments=3, new_length=1, modality='RGB',
                 image_tmpl='img_{:05d}.jpg', transform=None,
                 random_shift=True, test_mode=False,
                 remove_missing=False, dense_sample=False, twice_sample=False):

        self.root_path = root_path
        self.list_file = list_file
        self.num_segments = num_segments
        self.new_length = new_length
        self.modality = modality
        self.image_tmpl = image_tmpl
        self.transform = transform
        self.random_shift = random_shift
        self.test_mode = test_mode
        self.remove_missing = remove_missing
        self.dense_sample = dense_sample  # using dense sample as I3D
        self.twice_sample = twice_sample  # twice sample for more validation
        if self.dense_sample:
            print('=> Using dense sample for the dataset...')
        if self.twice_sample:
            print('=> Using twice sample for the dataset...')

        if self.modality == 'RGBDiff':
            self.new_length += 1  # Diff needs one more image to calculate diff

        self._parse_list()

    def _load_image(self, directory, idx):
        if self.modality == 'RGB' or self.modality == 'RGBDiff':
            try:
                print(os.path.join(self.root_path, directory, self.image_tmpl.format(idx)))
                return [Image.open(os.path.join(self.root_path, directory, self.image_tmpl.format(idx))).convert('RGB')]
            except Exception:
                print('error loading image:', os.path.join(self.root_path, directory, self.image_tmpl.format(idx)))
                return [Image.open(os.path.join(self.root_path, directory, self.image_tmpl.format(1))).convert('RGB')]
        elif self.modality == 'Flow':
            if self.image_tmpl == 'flow_{}_{:05d}.jpg':  # ucf
                x_img = Image.open(os.path.join(self.root_path, directory, self.image_tmpl.format('x', idx))).convert(
                    'L')
                y_img = Image.open(os.path.join(self.root_path, directory, self.image_tmpl.format('y', idx))).convert(
                    'L')
            elif self.image_tmpl == '{:06d}-{}_{:05d}.jpg':  # something v1 flow
                x_img = Image.open(os.path.join(self.root_path, '{:06d}'.format(int(directory)), self.image_tmpl.
                                                format(int(directory), 'x', idx))).convert('L')
                y_img = Image.open(os.path.join(self.root_path, '{:06d}'.format(int(directory)), self.image_tmpl.
                                                format(int(directory), 'y', idx))).convert('L')
            else:
                try:
                    # idx_skip = 1 + (idx-1)*5
                    flow = Image.open(os.path.join(self.root_path, directory, self.image_tmpl.format(idx))).convert(
                        'RGB')
                except Exception:
                    print('error loading flow file:',
                          os.path.join(self.root_path, directory, self.image_tmpl.format(idx)))
                    flow = Image.open(os.path.join(self.root_path, directory, self.image_tmpl.format(1))).convert('RGB')
                # the input flow file is RGB image with (flow_x, flow_y, blank) for each channel
                flow_x, flow_y, _ = flow.split()
                x_img = flow_x.convert('L')
                y_img = flow_y.convert('L')

            return [x_img, y_img]

    def _parse_list(self):
        # check the frame number is large >3:
        tmp = [x.strip().split(' ') for x in open(self.list_file)]
        if not self.test_mode or self.remove_missing:
            tmp_temp = []
            for item in tmp:
                dir_folder = os.path.join(self.root_path, item[0])
                num_frames = len(os.listdir(dir_folder))
                if num_frames >= self.num_segments:
                    print('before:', item[1])
                    item[1] = num_frames
                    print('after:', item[1])
                    tmp_temp.append(item)
            tmp = tmp_temp
#                 tmp = [item for item in tmp if int(item[1]) >= 3]
        self.video_list = [VideoRecord(item) for item in tmp]

        if self.image_tmpl == '{:06d}-{}_{:05d}.jpg':
            for v in self.video_list:
                v._data[1] = int(v._data[1]) / 2
        print('video number:%d' % (len(self.video_list)))

    def _sample_indices(self, record):
        """
        :param record: VideoRecord
        :return: list
        """
        if self.dense_sample:  # i3d dense sample
            sample_pos = max(1, 1 + record.num_frames - 64)
            t_stride = 64 // self.num_segments
            start_idx = 0 if sample_pos == 1 else np.random.randint(0, sample_pos - 1)
            offsets = [(idx * t_stride + start_idx) % record.num_frames for idx in range(self.num_segments)]
            return np.array(offsets) + 1
        else:  # normal sample
            average_duration = (record.num_frames - self.new_length + 1) // self.num_segments
            if average_duration > 0:
                offsets = np.multiply(list(range(self.num_segments)), average_duration) + randint(average_duration,
                                                                                                  size=self.num_segments)
            elif record.num_frames > self.num_segments:
                offsets = np.sort(randint(record.num_frames - self.new_length + 1, size=self.num_segments))
            else:
                offsets = np.zeros((self.num_segments,))
            return offsets + 1

    def _get_val_indices(self, record):
        if self.dense_sample:  # i3d dense sample
            sample_pos = max(1, 1 + record.num_frames - 64)
            t_stride = 64 // self.num_segments
            start_idx = 0 if sample_pos == 1 else np.random.randint(0, sample_pos - 1)
            offsets = [(idx * t_stride + start_idx) % record.num_frames for idx in range(self.num_segments)]
            return np.array(offsets) + 1
        else:
            if record.num_frames > self.num_segments + self.new_length - 1:
                tick = (record.num_frames - self.new_length + 1) / float(self.num_segments)
                offsets = np.array([int(tick / 2.0 + tick * x) for x in range(self.num_segments)])
            else:
                offsets = np.zeros((self.num_segments,))
            return offsets + 1

    def _get_test_indices(self, record):
        if self.dense_sample:
            sample_pos = max(1, 1 + record.num_frames - 64)
            t_stride = 64 // self.num_segments
            start_list = np.linspace(0, sample_pos - 1, num=10, dtype=int)
            offsets = []
            for start_idx in start_list.tolist():
                offsets += [(idx * t_stride + start_idx) % record.num_frames for idx in range(self.num_segments)]
            return np.array(offsets) + 1
        elif self.twice_sample:
            tick = (record.num_frames - self.new_length + 1) / float(self.num_segments)

            offsets = np.array([int(tick / 2.0 + tick * x) for x in range(self.num_segments)] +
                               [int(tick * x) for x in range(self.num_segments)])

            return offsets + 1
        else:
            tick = (record.num_frames - self.new_length + 1) / float(self.num_segments)
            offsets = np.array([int(tick / 2.0 + tick * x) for x in range(self.num_segments)])
            return offsets + 1

    def __getitem__(self, index):
        record = self.video_list[index]
        # check this is a legit video folder

        if self.image_tmpl == 'flow_{}_{:05d}.jpg':
            file_name = self.image_tmpl.format('x', 1)
            full_path = os.path.join(self.root_path, record.path, file_name)
        elif self.image_tmpl == '{:06d}-{}_{:05d}.jpg':
            file_name = self.image_tmpl.format(int(record.path), 'x', 1)
            full_path = os.path.join(self.root_path, '{:06d}'.format(int(record.path)), file_name)
        else:
            print('here1')
            file_name = self.image_tmpl.format(1)
            full_path = os.path.join(self.root_path, record.path, file_name)
            print('full_path: ',full_path)
        while not os.path.exists(full_path):
            print('################## Not Found:', os.path.join(self.root_path, record.path, file_name))
            index = np.random.randint(len(self.video_list))
            record = self.video_list[index]
            if self.image_tmpl == 'flow_{}_{:05d}.jpg':
                file_name = self.image_tmpl.format('x', 1)
                full_path = os.path.join(self.root_path, record.path, file_name)
            elif self.image_tmpl == '{:06d}-{}_{:05d}.jpg':
                file_name = self.image_tmpl.format(int(record.path), 'x', 1)
                full_path = os.path.join(self.root_path, '{:06d}'.format(int(record.path)), file_name)
            else:
                file_name = self.image_tmpl.format(1)
                full_path = os.path.join(self.root_path, record.path, file_name)

        if not self.test_mode:
            segment_indices = self._sample_indices(record) if self.random_shift else self._get_val_indices(record)
        else:
            segment_indices = self._get_test_indices(record)
        return self.get(record, segment_indices)

    def get(self, record, indices):

        images = list()
        for seg_ind in indices:
            p = int(seg_ind)
            for i in range(self.new_length):
                seg_imgs = self._load_image(record.path, p)
                images.extend(seg_imgs)
                if p < record.num_frames:
                    p += 1

#         process_data = self.transform(images)
        return images, record.label

    def __len__(self):
        return len(self.video_list)

In [53]:
root_path = '/workspace1/afvilla/meta-something-something-v2_frames'
list_file = '/mnt/nas2/GrimaRepo/afvilla/FewShotProject/data/data_s2s/somethingv2_meta_train_train.txt'

In [61]:
dataset = TSNDataSet(root_path, list_file,
                 num_segments=8, new_length=1, modality='RGB',
                 image_tmpl='{:01d}.jpg', transform=None,
                 random_shift=True, test_mode=False,
                 remove_missing=False, dense_sample=False, twice_sample=False)

before: 54
after: 52
before: 51
after: 49
before: 49
after: 48
before: 35
after: 35
before: 17
after: 17
before: 48
after: 47
before: 49
after: 48
before: 33
after: 33
before: 67
after: 63
before: 37
after: 37
before: 67
after: 63
before: 44
after: 44
before: 37
after: 37
before: 41
after: 41
before: 47
after: 36
before: 41
after: 41
before: 34
after: 34
before: 51
after: 49
before: 64
after: 60
before: 38
after: 38
before: 30
after: 30
before: 47
after: 46
before: 37
after: 37
before: 58
after: 55
before: 28
after: 28
before: 37
after: 37
before: 62
after: 59
before: 63
after: 59
before: 57
after: 54
before: 39
after: 39
before: 27
after: 27
before: 64
after: 60
before: 33
after: 33
before: 36
after: 36
before: 31
after: 31
before: 16
after: 16
before: 48
after: 47
before: 50
after: 49
before: 33
after: 33
before: 44
after: 44
before: 50
after: 49
before: 49
after: 48
before: 65
after: 61
before: 19
after: 19
before: 50
after: 49
before: 27
after: 27
before: 36
after: 36
before: 30
af

before: 30
after: 30
before: 61
after: 58
before: 56
after: 54
before: 36
after: 35
before: 46
after: 45
before: 25
after: 25
before: 57
after: 54
before: 58
after: 55
before: 42
after: 42
before: 53
after: 51
before: 62
after: 59
before: 40
after: 40
before: 57
after: 54
before: 36
after: 36
before: 23
after: 23
before: 54
after: 52
before: 47
after: 46
before: 40
after: 40
before: 64
after: 60
before: 37
after: 37
before: 50
after: 48
before: 59
after: 56
before: 59
after: 56
before: 59
after: 56
before: 65
after: 61
before: 71
after: 66
before: 51
after: 49
before: 68
after: 64
before: 58
after: 55
before: 28
after: 28
before: 37
after: 37
before: 63
after: 59
before: 51
after: 49
before: 51
after: 49
before: 30
after: 30
before: 53
after: 51
before: 25
after: 25
before: 32
after: 32
before: 44
after: 44
before: 37
after: 37
before: 59
after: 56
before: 39
after: 39
before: 41
after: 41
before: 36
after: 36
before: 42
after: 42
before: 61
after: 58
before: 44
after: 44
before: 43
af

before: 26
after: 26
before: 29
after: 29
before: 54
after: 35
before: 28
after: 28
before: 48
after: 47
before: 38
after: 38
before: 22
after: 22
before: 43
after: 43
before: 48
after: 47
before: 53
after: 51
before: 44
after: 44
before: 25
after: 25
before: 43
after: 41
before: 24
after: 23
before: 66
after: 62
before: 43
after: 43
before: 35
after: 35
before: 48
after: 47
before: 29
after: 29
before: 54
after: 52
before: 24
after: 24
before: 33
after: 33
before: 42
after: 42
before: 36
after: 36
before: 60
after: 57
before: 60
after: 53
before: 41
after: 41
before: 25
after: 25
before: 37
after: 37
before: 53
after: 51
before: 29
after: 29
before: 32
after: 32
before: 29
after: 29
before: 61
after: 58
before: 49
after: 48
before: 30
after: 30
before: 21
after: 21
before: 49
after: 48
before: 31
after: 31
before: 37
after: 37
before: 36
after: 36
before: 36
after: 36
before: 42
after: 42
before: 43
after: 43
before: 46
after: 45
before: 35
after: 35
before: 48
after: 47
before: 46
af

before: 39
after: 39
before: 40
after: 40
before: 31
after: 31
before: 59
after: 56
before: 42
after: 42
before: 28
after: 28
before: 60
after: 57
before: 59
after: 56
before: 38
after: 36
before: 62
after: 59
before: 51
after: 49
before: 38
after: 38
before: 39
after: 39
before: 49
after: 48
before: 39
after: 39
before: 41
after: 41
before: 45
after: 44
before: 52
after: 50
before: 43
after: 43
before: 43
after: 43
before: 51
after: 49
before: 60
after: 57
before: 41
after: 41
before: 39
after: 39
before: 38
after: 38
before: 48
after: 47
before: 31
after: 31
before: 36
after: 36
before: 54
after: 52
before: 50
after: 49
before: 33
after: 33
before: 63
after: 59
before: 43
after: 43
before: 40
after: 40
before: 46
after: 45
before: 37
after: 37
before: 50
after: 48
before: 39
after: 39
before: 53
after: 51
before: 47
after: 46
before: 63
after: 59
before: 44
after: 44
before: 38
after: 38
before: 67
after: 63
before: 44
after: 44
before: 56
after: 54
before: 50
after: 49
before: 66
af

before: 59
after: 56
before: 47
after: 46
before: 25
after: 25
before: 53
after: 51
before: 49
after: 48
before: 29
after: 29
before: 51
after: 49
before: 31
after: 31
before: 58
after: 55
before: 29
after: 29
before: 35
after: 35
before: 25
after: 25
before: 59
after: 56
before: 26
after: 26
before: 30
after: 30
before: 25
after: 25
before: 36
after: 36
before: 31
after: 31
before: 52
after: 50
before: 40
after: 40
before: 71
after: 66
before: 37
after: 37
before: 40
after: 40
before: 28
after: 28
before: 39
after: 39
before: 47
after: 46
before: 27
after: 27
before: 66
after: 62
before: 54
after: 52
before: 37
after: 37
before: 39
after: 39
before: 29
after: 29
before: 51
after: 32
before: 46
after: 45
before: 30
after: 30
before: 42
after: 42
before: 65
after: 61
before: 32
after: 32
before: 48
after: 46
before: 36
after: 36
before: 60
after: 57
before: 63
after: 59
before: 43
after: 43
before: 54
after: 52
before: 64
after: 60
before: 64
after: 60
before: 48
after: 47
before: 33
af

...

before: 68
after: 64
before: 68
after: 64
before: 33
after: 33
before: 34
after: 34
before: 50
after: 49
before: 62
after: 59
before: 36
after: 36
before: 60
after: 57
before: 36
after: 36
before: 45
after: 44
before: 68
after: 64
before: 28
after: 28
before: 67
after: 63
before: 33
after: 33
before: 28
after: 28
before: 46
after: 45
before: 31
after: 31
before: 41
after: 41
before: 39
after: 39
before: 49
after: 48
before: 26
after: 26
before: 68
after: 64
before: 47
after: 46
before: 55
after: 53
before: 35
after: 35
before: 47
after: 46
before: 60
after: 57
before: 53
after: 51
before: 49
after: 48
before: 33
after: 33
before: 47
after: 46
before: 42
after: 42
before: 42
after: 42
before: 53
after: 51
before: 45
after: 44
before: 40
after: 40
before: 44
after: 44
before: 50
after: 49
before: 27
after: 27
before: 53
after: 51
before: 61
after: 58
before: 37
after: 37
before: 72
after: 67
before: 29
after: 29
before: 43
after: 30
before: 43
after: 43
before: 59
after: 41
before: 24
af

before: 45
after: 44
before: 44
after: 44
before: 48
after: 47
before: 51
after: 49
before: 35
after: 35
before: 55
after: 53
before: 60
after: 57
before: 39
after: 39
before: 48
after: 47
before: 34
after: 34
before: 56
after: 54
before: 34
after: 34
before: 41
after: 41
before: 53
after: 51
before: 51
after: 49
before: 29
after: 29
before: 34
after: 34
before: 60
after: 57
before: 37
after: 37
before: 65
after: 61
before: 24
after: 24
before: 56
after: 54
before: 60
after: 57
before: 0
after: 32
before: 41
after: 41
before: 37
after: 37
before: 45
after: 44
before: 56
after: 54
before: 62
after: 59
before: 26
after: 26
before: 36
after: 36
before: 37
after: 37
before: 26
after: 26
before: 58
after: 55
before: 70
after: 65
before: 32
after: 32
before: 63
after: 59
before: 59
after: 56
before: 43
after: 43
before: 29
after: 29
before: 27
after: 27
before: 40
after: 40
before: 67
after: 63
before: 64
after: 60
before: 37
after: 37
before: 40
after: 40
before: 50
after: 49
before: 31
aft

before: 52
after: 50
before: 56
after: 54
before: 40
after: 40
before: 45
after: 44
before: 60
after: 57
before: 55
after: 53
before: 62
after: 59
before: 60
after: 57
before: 45
after: 44
before: 51
after: 48
before: 27
after: 27
before: 30
after: 30
before: 60
after: 57
before: 35
after: 35
before: 48
after: 47
before: 42
after: 42
before: 50
after: 49
before: 67
after: 63
before: 61
after: 58
before: 45
after: 44
before: 15
after: 15
before: 70
after: 65
before: 49
after: 35
before: 26
after: 26
before: 37
after: 37
before: 33
after: 33
before: 63
after: 59
before: 53
after: 51
before: 55
after: 53
before: 40
after: 40
before: 31
after: 31
before: 68
after: 64
before: 42
after: 42
before: 53
after: 51
before: 57
after: 54
before: 57
after: 54
before: 39
after: 39
before: 71
after: 43
before: 34
after: 34
before: 14
after: 14
before: 51
after: 49
before: 38
after: 38
before: 49
after: 48
before: 35
after: 35
before: 43
after: 43
before: 33
after: 33
before: 44
after: 44
before: 68
af

before: 42
after: 42
before: 47
after: 46
before: 40
after: 40
before: 38
after: 38
before: 67
after: 63
before: 22
after: 22
before: 45
after: 44
before: 58
after: 55
before: 58
after: 55
before: 29
after: 29
before: 39
after: 38
before: 54
after: 52
before: 40
after: 40
before: 26
after: 26
before: 41
after: 41
before: 46
after: 45
before: 27
after: 27
before: 35
after: 35
before: 52
after: 50
before: 45
after: 44
before: 48
after: 47
before: 31
after: 31
before: 23
after: 23
before: 45
after: 44
before: 54
after: 52
before: 45
after: 44
before: 34
after: 34
before: 33
after: 33
before: 28
after: 28
before: 45
after: 44
before: 31
after: 31
before: 36
after: 36
before: 37
after: 37
before: 35
after: 35
before: 28
after: 28
before: 58
after: 55
before: 51
after: 49
before: 41
after: 41
before: 44
after: 44
before: 43
after: 43
before: 34
after: 34
before: 39
after: 39
before: 51
after: 49
before: 26
after: 26
before: 34
after: 34
before: 43
after: 43
before: 45
after: 44
before: 32
af

before: 54
after: 52
before: 44
after: 44
before: 57
after: 54
before: 54
after: 52
before: 40
after: 40
before: 24
after: 24
before: 50
after: 49
before: 39
after: 39
before: 46
after: 45
before: 57
after: 54
before: 30
after: 30
before: 59
after: 55
before: 55
after: 53
before: 38
after: 38
before: 64
after: 60
before: 58
after: 55
before: 63
after: 59
before: 52
after: 50
before: 60
after: 57
before: 28
after: 28
before: 57
after: 54
before: 26
after: 26
before: 31
after: 31
before: 56
after: 54
before: 33
after: 33
before: 28
after: 28
before: 41
after: 41
before: 40
after: 40
before: 42
after: 42
before: 34
after: 34
before: 46
after: 45
before: 34
after: 34
before: 51
after: 49
before: 64
after: 60
before: 51
after: 49
before: 49
after: 34
before: 50
after: 49
before: 46
after: 45
before: 56
after: 54
before: 22
after: 20
before: 33
after: 33
before: 65
after: 61
before: 44
after: 44
before: 38
after: 38
before: 43
after: 43
before: 38
after: 20
before: 27
after: 27
before: 34
af

In [65]:
dataset.__getitem__(50)

here1
full_path:  /workspace1/afvilla/meta-something-something-v2_frames/198925/1.jpg
/workspace1/afvilla/meta-something-something-v2_frames/198925/3.jpg
/workspace1/afvilla/meta-something-something-v2_frames/198925/7.jpg
/workspace1/afvilla/meta-something-something-v2_frames/198925/14.jpg
/workspace1/afvilla/meta-something-something-v2_frames/198925/16.jpg
/workspace1/afvilla/meta-something-something-v2_frames/198925/23.jpg
/workspace1/afvilla/meta-something-something-v2_frames/198925/28.jpg
/workspace1/afvilla/meta-something-something-v2_frames/198925/34.jpg
/workspace1/afvilla/meta-something-something-v2_frames/198925/39.jpg


([<PIL.Image.Image image mode=RGB size=427x240 at 0x7F5067DFADD8>,
  <PIL.Image.Image image mode=RGB size=427x240 at 0x7F5067DFA940>,
  <PIL.Image.Image image mode=RGB size=427x240 at 0x7F5067E06080>,
  <PIL.Image.Image image mode=RGB size=427x240 at 0x7F5067E06128>,
  <PIL.Image.Image image mode=RGB size=427x240 at 0x7F5067E063C8>,
  <PIL.Image.Image image mode=RGB size=427x240 at 0x7F5067E06550>,
  <PIL.Image.Image image mode=RGB size=427x240 at 0x7F5067E066D8>,
  <PIL.Image.Image image mode=RGB size=427x240 at 0x7F5067E06828>],
 17)

In [58]:
!python model/temporalShiftModule/main.py something RGB \
     --arch resnet34 --num_segments 8 \
     --gd 20 --lr 0.01 --lr_steps 20 40 --epochs 50 \
     --batch-size 64 -j 16 --dropout 0.5 --consensus_type=avg --eval-freq=1 \
     --shift --shift_div=8 --shift_place=blockres --npb

Traceback (most recent call last):
  File "model/temporalShiftModule/main.py", line 14, in <module>
    from ops.dataset import TSNDataSet
  File "/mnt/nas2/GrimaRepo/afvilla/FewShotProject/model_cnaps/FewShotLearningForVideo/model/temporalShiftModule/ops/__init__.py", line 1, in <module>
    from model.temporalShiftModule.ops.basic_ops import *
ModuleNotFoundError: No module named 'model'
